# An Example of using `minimod`

For this notebook, we will first create the data needs to be used for the program. Then we will instantiate the class and make a report of the results using the `report` method

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import sys
import pandas as pd
import numpy as np
import minimod as mm
import mip

Using Python-MIP package version 1.6.8


In [3]:
## Data

# This is how the data was processed, assuming we use the Katie_VA_Benefits_and_Costs_1_8_2019.xlsx file.

## First we get a path to the raw data
excel_file = "data/raw/Katie_VA_Benefits_and_Costs_1_8_2019.xlsx"


# Get "vasoilold" and get discounted benefits for constraint

discount_costs = 1/(1 + 0.03)


vasoilold_constraint = (
    pd.read_excel(excel_file,
                  sheet_name = 'Benefits',
                  header = 2)
    .loc[lambda df : df['intervention'] == 'vasoilold']
    .set_index(['intervention', 'space'])
    .stack()
    .to_frame()
    .reset_index()
    .rename({
        'level_2' : 'time',
        0 : 'benefit'
    }, axis= 'columns')
    .assign(
        time_rank = lambda df: (df['time'].rank(numeric_only=True, method= 'dense') -1).astype(int) ,
        time_discount_costs = lambda df: discount_costs**df['time_rank'],
        discounted_benefits = lambda df: df['time_discount_costs']*df['benefit']
    )
    ['discounted_benefits'].sum()
    
)


In [4]:
df_benefit = (pd.read_excel(excel_file, 
                           sheet_name = 'Benefits',
                           header = 2)
              .loc[lambda df: df['intervention'] != 'vasoilold']
              .set_index(['intervention', 'space'])
              .stack()
              .to_frame()
              .reset_index()
              .rename({'level_2' : 'time',
                       0 : 'benefit'}, axis=1)
              .set_index(['intervention', 'space', 'time'])
              )

df_cost = (pd.read_excel(excel_file, 
                           sheet_name = 'Costs',
                           header = 2)
           .loc[lambda df: df['intervention'] != 'vasoilold']
              .set_index(['intervention', 'space'])
              .stack()
              .to_frame()
              .reset_index()
              .rename({'level_2' : 'time',
                       0 : 'costs'}, axis=1)
              .set_index(['intervention', 'space', 'time'])
              )

In [5]:
# Then we merge the cost and benefit data together.

df = (df_benefit
      .merge(df_cost, left_index=True, right_index=True)
      .reset_index()    
)


# Then we save the data. The finished data can be found in the `/examples/data/raw` folder.

df.to_csv("data/processed/example1.csv")


In [3]:
df = pd.read_csv("data/processed/example1.csv")

## Running the model

Now we instantiate the model, and then run `fit` and get the report.

In [28]:
c = mm.CostSolver(minimum_benefit = vasoilold_constraint)


              MiniMod Nutrition Intervention Tool
              Optimization Method: MIN
              Version: 0.0.1dev
              Solver: CBC
              
              


In [29]:
opt = c.fit(data = df, 
all_space = ['cube', 'oil', 'maize'], 
all_time = ['maize', 'cube'])

[Note]: Processing Data...
[Note]: Creating Base Model with constraints
[Note]: Optimizing...
[Note]: Optimal Solution Found


In [30]:
c.report()

opt_df = c.opt_df.reset_index()


+----------------------------+----------------------------+
| MiniMod Solver Results     |                            |
| Method:                    | MIN                        |
| Solver:                    | CBC                        |
| Optimization Status:       | OptimizationStatus.OPTIMAL |
| Number of Solutions Found: | 1                          |
+----------------------------+----------------------------+
+-----------------------------+-------+
| No. of Variables:           |   450 |
| No. of Integer Variables:   |   450 |
| No. of Constraints          |  5791 |
| No. of Non-zeros in Constr. | 12402 |
+-----------------------------+-------+
+-----------------+-------------+
| Minimum Benefit | 1.59582e+07 |
| Total Cost      | 1.97115e+07 |
| Total Coverage  | 2.06205e+07 |
+-----------------+-------------+
+----------------------------------+--+
| Total Costs and Coverage by Year |  |
+----------------------------------+--+
|   time |   opt_vals |   opt_benefit |   opt_cost

In [15]:
c.write()